
## Models from Scratch: Linear Regression

##### Implementing the fundamental functions used to fit a linear regression model. 
***

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [2]:
# Read in Credit data
df = pd.read_csv('./data/Credit.csv', index_col=0)
# Log-transform incomes (Generally good practice)
df.Income = np.log(df.Income)
df.head()

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
1,2.700757,3606,283,2,34,11,Male,No,Yes,Caucasian,333
2,4.663675,6645,483,3,82,15,Female,Yes,Yes,Asian,903
3,4.650077,7075,514,4,71,11,Male,No,No,Asian,580
4,5.003436,9504,681,3,36,11,Female,No,No,Asian,964
5,4.023242,4897,357,2,68,16,Male,No,Yes,Caucasian,331


In [3]:
# For now, let's look only at the quantitative predictors. We will use these to predict income.
df2 = df[['Limit', 'Rating', 'Cards', 'Age', 'Education']].copy()

In [4]:
# column vector of 1s (needed if we want an intercept term)
ones = np.full((len(df2),), 1, dtype=int)
df2.insert(0, 'Ones', ones)

In [5]:
# We now have our matrix of data X
X = np.asarray(df2)
# And our vector of outputs Y. Note that the income column uses periods, not commas
# Values will be interpreted as floats
y = np.asarray(df['Income'])

### Fitting The Model: First Approach
#### In this first approach, we estimate the coefficients by algorithmically minimizing the residual sum of squares.

In [6]:
def RSS(b):
    """Residual sum of squares as a function of the beta vector"""
    return np.linalg.norm(y - np.dot(X, b))**2

In [7]:
def fit(X, y):
    """Returns the vector of beta estimates in linear regression given design matrix X and output vector y"""
    # To estimate the coefficients (betas), we minimize RSS with respect to beta.
    guess = np.zeros((6, ))
    min = minimize(RSS, guess)
    return min.x

In [8]:
betas = fit(X, y)
print(list(betas))

[2.4868600598268422, 0.00017000883748685867, 0.00067429958418413428, -0.033626071064926981, 0.0038989089959176543, -0.0063675285935832875]


So the linear regression function in this example is Y = -22.31+ 0.0023X<sub>1</sub> +  0.1449X<sub>2</sub> -  1.4973X<sub>3</sub> + 0.1987X<sub>4</sub> - 0.0942X<sub>5</sub>, where each X<sub>i</sub> corresponds to the predictors 'Limit', 'Rating', 'Cards', 'Age', and 'Education'.

### Fitting The Model: Second Approach

#### The first method works fine; however, OLS estimation leads to a simple, closed-form solution to the minimization problem. Thus, there's no need to utilize an optimization algorithm. In general, this is not possible in other statistical learning methods. Specifically, the residual sum of squares is minimized by : b = (X<sup>T</sup>X)<sup>-1</sup>X<sup>T</sup>Y.                                       Let A = (X<sup>T</sup>X)<sup>-1</sup> and  C = </sup>X<sup>T</sup>Y. Thus, b = AC

In [9]:
def fit2(X, y):
    """Returns the vector of beta estimates in linear regression given design matrix X and output vector y"""
    X_trans = np.transpose(X)
    A = np.linalg.inv(np.dot(X_trans, X))
    C = np.dot(X_trans, y)
    return np.dot(A, C)

In [10]:
betas = list(fit2(X,y))
print(betas)

[2.4899794482709297, 0.00017819003055282844, 0.00055237986324040145, -0.033037864075505879, 0.0038993761245820872, -0.0063938246477528438]


And we can see that both approaches resulted in the same estimated coefficients. 

### A basic Linear Regression Class

In [11]:
class LinearRegression():

    def __init__(self, X, y):
        """Initialize a Linear Regression object"""
        if len(X) != len(y):
            raise ValueError("Dimensional error: {} != {}".format(len(X), len(y)))
        self.X = np.asarray(X)
        self.y = np.asarray(y)
        self.n = len(X)  # Number of observations
        self.p = len(X[0])  # Number of predictors
        self.coefficients = None  # Coefficients to be estimated

    def fit(self):
        """Fits the data to the linear regression model by estimating the coefficients."""
        X_trans = np.transpose(self.X)
        A = np.linalg.inv(np.dot(X_trans, self.X))
        C = np.dot(X_trans, self.y)

        # Update estimated coefficients
        self.coefficients = np.dot(A, C)

    def predict(self, x):
        """Predicts y given an observation vector x"""
        y = 0
        for term in list(zip(self.coefficients, x)):
            y += term[0]*term[1]
        return y

In [12]:
model = LinearRegression(X, y)

In [13]:
model.fit()

In [14]:
list(model.coefficients) == betas

True

In [15]:
print("Predicted value for the first training observation: {}".format(model.predict([1, 3606, 283, 2, 34, 11])))
print("Actual value for the first training observation: {}".format(y[0]))

Predicted value for the first training observation: 3.2850271887009606
Actual value for the first training observation: 2.700757003608068


***